In [1]:
using ArgParse
using BSON
using Distributions
using Flux
using Stheno
using Tracker
#using CUDA


In [5]:

include("NeuralProcesses.jl/src/NeuralProcesses.jl")
using .NeuralProcesses


┌ Warning: Replacing docs for `Main.NeuralProcesses.Categorical :: Union{}` in module `Main.NeuralProcesses`
└ @ Base.Docs docs/Docs.jl:240


In [ ]:

# parser = ArgParseSettings()
# @add_arg_table! parser begin
#     "--gen"
#         help = "Experiment setting: gridworld, menu_search, h_menu_search"
#         arg_type = String
#         default = "menu_search"
#     "--n_traj"
#         help = "Number of context trajectories. Setting to 0 randomizes between 1 and 8."
#         arg_type = Int
#         default = 0
#     "--n_epochs"
#         help = "Number of training epochs."
#         arg_type = Int
#         default = 50
#     "--n_batches"
#         help = "Number of batches."
#         arg_type = Int
#         default = 25
#     "--batch_size"
#         help = "Batch size."
#         arg_type = Int
#         default = 4
#     "--params"
#         help = "Return params?"
#         arg_type = Bool
#         default = false
#     "--p_bias"
#         help = "Probability of generating a sample with biased model"
#         arg_type = Float64
#         default = 0.0
#     "--bson"
#         help = "Directly specify the file to save the model to and load it from."
#         arg_type = String
#     "--epsilon"
#         help = "Value for epsilon."
#         arg_type = Float64
# end
# args = parse_args(parser)

In [12]:
function get_default_args()
    defaults = Dict(
        "gen" => "menu_search",
        "n_traj" => 0,
        "n_epochs" => 50,
        "n_batches" => 25,
        "batch_size" => 4,
        "params" => false,
        "p_bias" => 0.0,
        "bson" => "",
        "epsilon" => 0.0
    )
    return defaults
end

args = get_default_args()

Dict{String, Any} with 9 entries:
  "p_bias"     => 0.0
  "bson"       => ""
  "gen"        => "menu_search"
  "epsilon"    => 0.0
  "batch_size" => 4
  "n_epochs"   => 50
  "n_batches"  => 25
  "params"     => false
  "n_traj"     => 0

In [7]:
# println("Initializing model...")

# model = anp_ex2(
#     dim_embedding=128,
#     num_encoder_heads=8,
#     num_encoder_layers=6,
#     num_decoder_layers=6,
#     args=args
# ) |> gpu


Initializing model...


┌ Info: The GPU function is being called but the GPU is not accessible. 
└ Defaulting back to the CPU. (No action is required if you want to run on the CPU).


Model(Main.NeuralProcesses.Parallel{3}(Chain[Chain(InputsCoder(), DeterministicLikelihood()), Chain(Main.NeuralProcesses.Attention(Chain(Main.NeuralProcesses.BatchedMLP(Chain(Dense(17, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128)), 128), #117), Chain(Main.NeuralProcesses.BatchedMLP(Chain(Dense(26, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128, #84), Dense(128, 128)), 128), #118), Chain(_compress_channels, Main.NeuralProcesses.BatchedMLP(Chain(Dense(128, 128)), 128)), Main.NeuralProcesses.Transformer(Chain(_compress_channels, Main.NeuralProcesses.BatchedMLP(Chain(Dense(128, 128)), 128)), Main.NeuralProcesses.LayerNorm(Float32[1.0 1.0 … 1.0 1.0], Float32[0.0 0.0 … 0.0 0.0], (2,)), Main.NeuralProcesses.BatchedMLP(Chain(Dense(128, 128, #84), Dense(128, 128)), 128), Main.NeuralProcesses.LayerNorm(Float32[1.0 1.0 … 1.0 1.0], Float32[0.0 0.0 … 0.0 0.0], (2,)))), DeterministicLik

In [8]:
# println("Initializing loss...")

# loss(xs...) = np_elbo(
#     xs...,
#     num_samples=5,
#     fixed_σ_epochs=3
# )

Initializing loss...


loss (generic function with 1 method)

In [9]:
batch_size  = args["batch_size"]

# Redundant. Required to fit the DataGenerator definition
x_context = Distributions.Uniform(-2, 2)
x_target  = Distributions.Uniform(-2, 2)

num_context = Distributions.DiscreteUniform(10, 10)
num_target  = Distributions.DiscreteUniform(10, 10)

data_gen = NeuralProcesses.DataGenerator(
                SearchEnvSampler(args;),
                batch_size=batch_size,
                x_context=x_context,
                x_target=x_target,
                num_context=num_context,
                num_target=num_target,
                σ²=1e-8
            )

DataGenerator(SearchEnvSampler(Dict{String, Any}("p_bias" => 0.0, "bson" => "", "gen" => "menu_search", "batch_size" => 4, "params" => false, "n_traj" => 0), Beta{Float64}(α=3.0, β=1.35), Truncated(Normal{Float64}(μ=3.0, σ=1.0); lower=0.0, upper=5.0), Truncated(Normal{Float64}(μ=0.3, σ=0.3); lower=0.0, upper=1.0)), 4, Uniform{Float64}(a=-2.0, b=2.0), Uniform{Float64}(a=-2.0, b=2.0), DiscreteUniform(a=10, b=10), DiscreteUniform(a=10, b=10), 1.0e-8)

In [10]:
data = gen_batch(data_gen,1)

1-element Vector{Vector{Array{Float32, 3}}}:
 [[0.0 0.0 … 0.0 8.0; 2.0 2.0 … 2.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0]

[0.0 0.0 … 0.0 8.0; 3.0 2.0 … 2.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0]

[0.0 0.0 … 0.0 8.0; 1.0 1.0 … 1.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0]

...

[0.0 0.0 … 0.0 8.0; 2.0 2.0 … 2.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0]

[0.0 0.0 … 0.0 8.0; 2.0 2.0 … 2.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0]

[0.0 0.0 … 0.0 8.0; 0.0 0.0 … 0.0 4.0; … ; -1.0 -1.0 … -1.0 -1.0; 0.0 0.0 … 0.0 8.0], [0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]

[0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0]

[0.0 0.0 … 0.0 0.

In [13]:
# println("Proceeding to training loop...")

# mkpath("models/"*string(args["bson"]))

# train_model!(
#         model,
#         loss,
#         data_gen,
#         ADAM(5e-4),
#         bson=args["bson"],
# 	experiment=args["gen"],
#         starting_epoch=0,
#         tasks_per_epoch=2^5,
#         batches=args["n_batches"],
# 	total_epochs=args["n_epochs"],
#         epsilon=args["epsilon"]
#     )

Proceeding to training loop...


LoadError: UndefVarError: CUDA not defined